In [2]:
import json
import cobra
from escher import Builder
import pandas
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')
sns.set_context('notebook', font_scale = 1.5, rc = {'lines.linewidth' : 2.5})
plt.style.use('fivethirtyeight')
from cobra import Model, Reaction, Metabolite
%matplotlib inline

In [3]:
ecoli_core = cobra.io.read_sbml_model('./e_coli_core.xml')

In [22]:
ecoli_core_model = ecoli_core.copy()

In [4]:
print(len(ecoli_core.reactions)) #Number of reaction
print(len(ecoli_core.metabolites))
print(len(ecoli_core.genes))
print(ecoli_core.medium) #Thinks e coli can uptake millimolar per dry weight per hour

95
72
137
{'EX_co2_e': 1000.0, 'EX_glc__D_e': 10.0, 'EX_h_e': 1000.0, 'EX_h2o_e': 1000.0, 'EX_nh4_e': 1000.0, 'EX_o2_e': 1000.0, 'EX_pi_e': 1000.0}


In [5]:
print(ecoli_core.objective)
obj_fnc = ecoli_core.reactions.get_by_id('BIOMASS_Ecoli_core_w_GAM')
print(obj_fnc.reaction)

Maximize
1.0*BIOMASS_Ecoli_core_w_GAM - 1.0*BIOMASS_Ecoli_core_w_GAM_reverse_712e5
1.496 3pg_c + 3.7478 accoa_c + 59.81 atp_c + 0.361 e4p_c + 0.0709 f6p_c + 0.129 g3p_c + 0.205 g6p_c + 0.2557 gln__L_c + 4.9414 glu__L_c + 59.81 h2o_c + 3.547 nad_c + 13.0279 nadph_c + 1.7867 oaa_c + 0.5191 pep_c + 2.8328 pyr_c + 0.8977 r5p_c --> 59.81 adp_c + 4.1182 akg_c + 3.7478 coa_c + 59.81 h_c + 3.547 nadh_c + 13.0279 nadp_c + 59.81 pi_c


In [6]:
display(ecoli_core.reactions.get_by_id('EX_glc__D_e'))

Reaction identifier,EX_glc__D_e
Name,D-Glucose exchange
Memory address,0x26a66e13f40
Stoichiometry,glc__D_e <=> D-Glucose <=>
GPR,
Lower bound,-10.0
Upper bound,1000.0


In [7]:
solution = ecoli_core.optimize()
display(solution)

,fluxes,reduced_costs
PFK,7.477382,-1.387779e-17
PFL,0.000000,2.081668e-17
PGI,4.860861,0.000000e+00
PGK,-16.023526,6.938894e-18
PGL,4.959985,1.517883e-17
...,...,...
NADH16,38.534610,0.000000e+00
NADTRHD,0.000000,-2.546243e-03
NH4t,4.765319,-1.387779e-17
O2t,21.799493,0.000000e+00


In [8]:
ecoli_core.summary()
display(ecoli_core.metabolites.atp_c.summary())

Percent,Flux,Reaction,Definition
66.58%,45.51,ATPS4r,adp_c + 4.0 h_e + pi_c <=> atp_c + h2o_c + 3.0 h_c
23.44%,16.02,PGK,3pg_c + atp_c <=> 13dpg_c + adp_c
2.57%,1.758,PYK,adp_c + h_c + pep_c --> atp_c + pyr_c
7.41%,5.064,SUCOAS,atp_c + coa_c + succ_c <=> adp_c + pi_c + succoa_c
Percent,Flux,Reaction,Definition
12.27%,-8.39,ATPM,atp_c + h2o_c --> adp_c + h_c + pi_c
76.46%,-52.27,BIOMASS_Ecoli_core_w_GAM,1.496 3pg_c + 3.7478 accoa_c + 59.81 atp_c + 0.361 e4p_c + 0.0709 f6p_c + 0.129 g3p_c + 0.205 g6p_c + 0.2557 gln__L_c + 4.9414 glu__L_c + 59.81 h2o_c + 3.547 nad_c + 13.0279 nadph_c + 1.7867 oaa_c + 0.5191 pep_c + 2.8328 pyr_c + 0.8977 r5p_c --> 59.81 adp_c + 4.1182 akg_c + 3.7478 coa_c + 59.81 h_c + 3.547 nadh_c + 13.0279 nadp_c + 59.81 pi_c
0.33%,-0.2235,GLNS,atp_c + glu__L_c + nh4_c --> adp_c + gln__L_c + h_c + pi_c
10.94%,-7.477,PFK,atp_c + f6p_c --> adp_c + fdp_c + h_c


In [9]:
ecoli_core.reactions.EX_o2_e.bounds = [0,1000]
display(ecoli_core.reactions.EX_o2_e)

Reaction identifier,EX_o2_e
Name,O2 exchange
Memory address,0x26a66e215b0
Stoichiometry,o2_e --> O2 O2 -->
GPR,
Lower bound,0
Upper bound,1000


In [10]:
solution = ecoli_core.optimize()
display(solution)
print([s.id for s in ecoli_core.metabolites])
display(ecoli_core.metabolites.o2_c.summary())

,fluxes,reduced_costs
PFK,9.789459e+00,0.000000e+00
PFL,1.780467e+01,-3.469447e-18
PGI,9.956609e+00,0.000000e+00
PGK,-1.943734e+01,-0.000000e+00
PGL,0.000000e+00,-4.615013e-03
...,...,...
NADH16,0.000000e+00,-5.538015e-03
NADTRHD,0.000000e+00,-1.107603e-02
NH4t,1.154156e+00,-1.301043e-17
O2t,-2.220446e-15,-0.000000e+00


['glc__D_e', 'gln__L_c', 'gln__L_e', 'glu__L_c', 'glu__L_e', 'glx_c', 'h2o_c', 'h2o_e', 'h_c', 'h_e', 'icit_c', 'lac__D_c', 'lac__D_e', 'mal__L_c', 'mal__L_e', 'nad_c', 'nadh_c', 'nadp_c', 'nadph_c', 'nh4_c', '13dpg_c', 'nh4_e', 'o2_c', '2pg_c', 'o2_e', '3pg_c', 'oaa_c', 'pep_c', '6pgc_c', 'pi_c', '6pgl_c', 'pi_e', 'ac_c', 'pyr_c', 'pyr_e', 'q8_c', 'q8h2_c', 'r5p_c', 'ru5p__D_c', 'ac_e', 'acald_c', 's7p_c', 'acald_e', 'accoa_c', 'succ_c', 'succ_e', 'succoa_c', 'acon_C_c', 'xu5p__D_c', 'actp_c', 'adp_c', 'akg_c', 'akg_e', 'amp_c', 'atp_c', 'cit_c', 'co2_c', 'co2_e', 'coa_c', 'dhap_c', 'e4p_c', 'etoh_c', 'etoh_e', 'f6p_c', 'fdp_c', 'for_c', 'for_e', 'fru_e', 'fum_c', 'fum_e', 'g3p_c', 'g6p_c']


Percent,Flux,Reaction,Definition
Percent,Flux,Reaction,Definition


In [25]:
# save model in JSON to communicate with escher viewer and create the new map
# TO UPLOAD AND VISUALIZE RESULTS ON ESCHER (https://escher.github.io/).
cobra.io.save_json_model(ecoli_core_model, 'ecoli_core_model.json')
with open('ecoli_core_EX_glc__D_e_reactions.json', 'w') as outfile:  
    json.dump(solution.fluxes.to_dict(), outfile)
with open('ecoli_core_EX_glc__D_e_metabolites.json', 'w') as outfile:  
    json.dump(solution.shadow_prices.to_dict(), outfile)
# END JSON CREATION

# save model in JSON to communicate with escher viewer and create the new map
# TO WRITE JSON --> cobra.io.write_sbml_model(emodel, "model.xml")
cobra.io.save_json_model(ecoli_core_model, 'ecoli_core_model_ver1.json')

In [24]:
import escher
import escher.urls
def draw_map(model, solution):
            b = Builder(map_json="./ecoli_core_model.Core metabolism.json" ,# Change with the path on your PC.
            model=model,
            reaction_data=solution.fluxes.to_dict(),
            reaction_scale=[{'type': 'min', 'color': '#cccccc', 'size': 4},
                                   {'type': 'mean', 'color': '#0000dd', 'size': 20},
                                   {'type': 'max', 'color': '#ff0000', 'size': 40}],
            metabolite_data=solution.shadow_prices.to_dict(),
            hide_secondary_metabolites=False)

            return b

In [26]:
draw_map(ecoli_core, solution)

ValueError: Could not load map_json. Not valid json, url, or filepath